In [12]:
import tensorflow as tf
import numpy as np

# TensorFlow low-level API

The tensorflow API revolves around Tensors that flow from one operation to another: hence the name tensorflow. You can use TensorFlow in a manner very similar to numpy. A tf tensor is similar to a nn ndarray. 

In [3]:
# create a tensor as follows
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
# Tensors support all the common operations that you would expect in numpy
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [7]:
t[1, 1]

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
tf.transpose(t)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 4.],
       [2., 5.],
       [3., 6.]], dtype=float32)>

In [11]:
# tensors can also hold constant values
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

## Tensors and Numpy
Tensors play nicely with numpy

In [13]:
a = np.array([2,4,5])

In [16]:
t = tf.constant(a)
t

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 5])>

In [19]:
t.numpy()

array([2, 4, 5])

In [20]:
np.array(t)

array([2, 4, 5])

In [21]:
np.square(t)

array([ 4, 16, 25])

'''
Note that numpy defaults to 64-bit precision while TF defaults to 32-bit, which is 
more than enough for neural networks. So always specify i32 when you swap from np to tf.
'''

## Type Conversions
Tensorflow does not perform any type conversions automatically, as they can degrade performance and can easily go unnoticed.
- You cant add tensors with differing data types, eg f32 and f64, or i32 and f32

In [23]:
tf.constant(2) + tf.constant(2.)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:AddV2]

In [31]:
# Remember that the above is for a good cause! You can always case your tensor to other dtypes
t = tf.constant(2)
t2 = tf.constant(10.)
t + tf.cast(t2, dtype=tf.int32)

<tf.Tensor: shape=(), dtype=int32, numpy=12>

## Variables
thus far all the values have been immutable. Regular tensors thus cannot be used to implement weights and biases in a network. this is where tf.Variable comes in!

In [33]:
v = tf.Variable([[1,2,3], [4,5,6]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

Variables behave much the same as the constant tensors, but can be changed via .assign() or .assign_add() or .assign_sub()

In [35]:
v.assign(v*2)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 10, 12]])>

In [38]:
v[0,1] = 42

TypeError: 'ResourceVariable' object does not support item assignment

In [36]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2, 42,  6],
       [ 8, 10, 12]])>

In [39]:
v[:,2].assign([0,1])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2, 42,  0],
       [ 8, 10,  1]])>

you will however notice that you don't often need to use the assign functionality as Keras often takes care of this for you.

# Customizing Models and Training Algorithms
We start by making a custom loss function, which is a common use case. We will implement the Huber loss function. We write a function that takes the labels and the models predictions as arguments, and uses tensorflow operations to compute a tensor containing all the losses

In [40]:
def huber(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# you would then go as follows
# model.compile(loss=huber, optimizer='nadam')
# model.fit(...)

'''
But what happens to the custom loss once you save the model?
'''

## Saving and Loading Models wit Custom Components
Saving works fine. When you load a model with custom components, you must provide a directory that maps the function name to the actual function implementation.